In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(20,15)})
sns.set_context("talk")

Disclaimer : The dataset contains more than 3M posts of reddit, because there is to much data to proceed and considering we have not a tremoundous amount of computentional power we choose to select a subset of this dataset. 

We considered subsection of reddit that may be interesting to learn from, we excluded all subreddit who were about videogames. Plus we selected 300 k message, and summary.


This is the RAW dataset, no need to load it again. You can go directly to section : 'Summarization task' where we load a clean dataset

In [ ]:
reddit300k_csv ='/content/gdrive/MyDrive/MasterThesis/reddit300k_reed.csv'
# reddit300k_csv = "/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/reddit300k_reed.csv"
import pandas as pd

redd300k = df = pd.read_csv(reddit300k_csv, thousands  = ',')
redd300k_dropped = redd300k.drop("Unnamed: 0",axis = 1)
df = redd300k_dropped
redd300k_dropped


,content,subreddit,summary
0,"b'very late to the party, but this is a good o...",b'AskReddit',b'gramps lost his teeth at the beach and I fou...
1,"b""Almost two weeks ago now, I lost my job, whi...",b'trees',"b'Lost my job, friend now coming by twice a da..."
2,"b""I have a friend who has a bad habit of eatin...",b'AskReddit',b'Stealing makes you shop better'
3,b'There was once an average looking girl that ...,b'AskMen',b'Playfully tease me \n edit- words'
4,"b""these stories are a dime a dozen. currently ...",b'videos',b'COMCAST CAN SUCK A DICK'
...,...,...,...
164317,"b""I heard an argument on this once, in a magaz...",b'science',b'Might be worth considering how many (and how...
164318,b'I will tell you the truth. The first time I ...,b'gaming',"b'I think I hate KotOR as well, but I played t..."
164319,b'Romney left Bain in 1999... that\'s a VERY o...,b'politics',"b""So far everything you've written is wrong an..."
164320,"b'Late, but here\'s mine: \n When I was about ...",b'AskReddit',"b""Was blamed for something completly random th..."


#Analysis

In [ ]:
# Download additional resources from NLTK text processin library
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
cat = len(df.select_dtypes(include=['object']).columns)
num = len(df.select_dtypes(include=['int64','float64']).columns)
print('Total Features: ', cat, 'categorical', '+',
      num, 'numerical', '=', cat+num, 'features')


Total Features:  3 categorical + 0 numerical = 3 features


In [ ]:
df.isna().sum()

content      0
subreddit    0
summary      0
dtype: int64

In [ ]:
df[df.isna().any(axis=1)]

,content,subreddit,summary


In [ ]:
print(str(df.sample()['content']))

127622    b"So this is my first post and I need help. \n...
Name: content, dtype: object


In [ ]:
print(str(df.sample()['content']), end='\r')


1722    b"Jealousy is never really an emotion that res...


In [ ]:
df.iloc[0]['content']

'b\'very late to the party, but this is a good one! \\n I was visiting my grandparents in Newfoundland 2 summers ago.  We went to the beach with my 3 younger cousins.  We swam, ate a picnic and just had wholesome family fun.  I snuck a ciggy while the family was frolicking in the surf. I had no where to pitch my butt so I buried it.  On the drive home my granddad realizes that his partial denture isn\\\'t in his mouth.  I just look at him and say "I\\\'m the finder of things, turn around"  We drive back to the beach, tide is coming in.  I run to where we were sitting (my granddad`s walking stick is still there mostly buried in drifting sand).  I sink to my knees, start scrambling through the sand, find the burried butt.  Triangulate where everyone was sitting during lunch.  As the tide was licking my toes and my granddad was cursing that it was gone, I pulled the denture out of the sand.  I nonchalantly handed it to him and said "keep this in your mouth"\''

In [ ]:
df.iloc[0]['summary']

"b'gramps lost his teeth at the beach and I found them an hour later minutes before the tide swept them away \\n EDIT: formatting'"

# Additional Cleaning (Troll)


## First we clean the " b' " every string has at the begining

In [ ]:
df = df.astype(str)
df

,content,subreddit,summary
0,"b'very late to the party, but this is a good o...",b'AskReddit',b'gramps lost his teeth at the beach and I fou...
1,"b""Almost two weeks ago now, I lost my job, whi...",b'trees',"b'Lost my job, friend now coming by twice a da..."
2,"b""I have a friend who has a bad habit of eatin...",b'AskReddit',b'Stealing makes you shop better'
3,b'There was once an average looking girl that ...,b'AskMen',b'Playfully tease me \n edit- words'
4,"b""these stories are a dime a dozen. currently ...",b'videos',b'COMCAST CAN SUCK A DICK'
...,...,...,...
164317,"b""I heard an argument on this once, in a magaz...",b'science',b'Might be worth considering how many (and how...
164318,b'I will tell you the truth. The first time I ...,b'gaming',"b'I think I hate KotOR as well, but I played t..."
164319,b'Romney left Bain in 1999... that\'s a VERY o...,b'politics',"b""So far everything you've written is wrong an..."
164320,"b'Late, but here\'s mine: \n When I was about ...",b'AskReddit',"b""Was blamed for something completly random th..."


In [ ]:
df1 = df[['content','subreddit','summary']].astype(str)


In [ ]:
df1

,content,subreddit,summary
0,"b'very late to the party, but this is a good o...",b'AskReddit',b'gramps lost his teeth at the beach and I fou...
1,"b""Almost two weeks ago now, I lost my job, whi...",b'trees',"b'Lost my job, friend now coming by twice a da..."
2,"b""I have a friend who has a bad habit of eatin...",b'AskReddit',b'Stealing makes you shop better'
3,b'There was once an average looking girl that ...,b'AskMen',b'Playfully tease me \n edit- words'
4,"b""these stories are a dime a dozen. currently ...",b'videos',b'COMCAST CAN SUCK A DICK'
...,...,...,...
164317,"b""I heard an argument on this once, in a magaz...",b'science',b'Might be worth considering how many (and how...
164318,b'I will tell you the truth. The first time I ...,b'gaming',"b'I think I hate KotOR as well, but I played t..."
164319,b'Romney left Bain in 1999... that\'s a VERY o...,b'politics',"b""So far everything you've written is wrong an..."
164320,"b'Late, but here\'s mine: \n When I was about ...",b'AskReddit',"b""Was blamed for something completly random th..."


In [ ]:
list = ['content','subreddit','summary']
for i in list:
  df1[i] = df1[i].str[2:-1]

,content,subreddit,summary
0,"very late to the party, but this is a good one...",AskReddit,gramps lost his teeth at the beach and I found...
1,"Almost two weeks ago now, I lost my job, which...",trees,"Lost my job, friend now coming by twice a day ..."
2,I have a friend who has a bad habit of eating ...,AskReddit,Stealing makes you shop better
3,There was once an average looking girl that I ...,AskMen,Playfully tease me \n edit- words
4,these stories are a dime a dozen. currently I'...,videos,COMCAST CAN SUCK A DICK
...,...,...,...
164317,"I heard an argument on this once, in a magazin...",science,Might be worth considering how many (and how s...
164318,I will tell you the truth. The first time I pl...,gaming,"I think I hate KotOR as well, but I played thr..."
164319,Romney left Bain in 1999... that\'s a VERY ope...,politics,So far everything you've written is wrong and ...
164320,"Late, but here\'s mine: \n When I was about 6-...",AskReddit,Was blamed for something completly random that...


In [ ]:
df1.to_csv (r'/content/gdrive/MyDrive/MasterThesis/reddit300k_clean.csv', header=True, index = False)

## Trying to supress the trolls

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

This code compares if there is more than 5 similar important words (we suppressed the stop words) between the content and the summary

In [ ]:
def test_troll(x):
  listeuh = ['content','summary']
  listoh = []
  for k in listeuh :
  # split into words
    tokens = word_tokenize(x[k])
  #get words in lower case
    words = [word.lower() for word in tokens]
  # remove all tokens that are not alphabetic
    words = [word for word in words if word.isalpha()]
  # remove stop words
    nostop = [word for word in words if word not in stop_words]
 # we add all the "meaningful" list of words of the content and summary in listoh
    listoh.append(nostop)
  i = len(listoh[0] and listoh[1])
  return i>5

In [ ]:
df_no_troll = df1.loc[df.apply(test_troll, axis = 1)]
df_no_troll

,content,subreddit,summary
0,"very late to the party, but this is a good one...",AskReddit,gramps lost his teeth at the beach and I found...
1,"Almost two weeks ago now, I lost my job, which...",trees,"Lost my job, friend now coming by twice a day ..."
5,Most major governments actions are to insure t...,explainlikeimfive,The only way you can think Snowden is a traito...
6,"Hey /fitness. 18m, 6', roughly 164 lbs, recent...",Fitness,want to get rid of belly and build a better bo...
7,How many pop artists even still think of music...,videos,I guess he's unique/deep if you compare him to...
...,...,...,...
164317,"I heard an argument on this once, in a magazin...",science,Might be worth considering how many (and how s...
164318,I will tell you the truth. The first time I pl...,gaming,"I think I hate KotOR as well, but I played thr..."
164319,Romney left Bain in 1999... that\'s a VERY ope...,politics,So far everything you've written is wrong and ...
164320,"Late, but here\'s mine: \n When I was about 6-...",AskReddit,Was blamed for something completly random that...


In [ ]:
# df_no_troll.to_csv (r'/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/reddit300k_without_troll.csv', header=True, index = False)
df_no_troll.to_csv (r'/content/gdrive/MyDrive/MasterThesis/reddit300k_without_troll.csv', header=True, index = False)

# Importint the T5 - Summarizer

In [ ]:
!pip install transformers==2.8.0
!pip install torch
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

     |████████████████████████████████| 573kB 2.1MB/s 
     |████████████████████████████████| 133kB 4.1MB/s 
     |████████████████████████████████| 5.6MB 4.2MB/s 
     |████████████████████████████████| 901kB 25.0MB/s 
     |████████████████████████████████| 1.2MB 26.3MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 7.6MB 27.4MB/s 
ERROR: botocore 1.20.82 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


We try the T5 on one content to see if it works

In [ ]:
text =df_no_troll.iloc[0]['content']
preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: " + preprocess_text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("Original content : \n",preprocess_text)
print ("\nSummarized by algo text: \n",output)

Original content : 
 very late to the party, but this is a good one! \n I was visiting my grandparents in Newfoundland 2 summers ago.  We went to the beach with my 3 younger cousins.  We swam, ate a picnic and just had wholesome family fun.  I snuck a ciggy while the family was frolicking in the surf. I had no where to pitch my butt so I buried it.  On the drive home my granddad realizes that his partial denture isn\'t in his mouth.  I just look at him and say "I\'m the finder of things, turn around"  We drive back to the beach, tide is coming in.  I run to where we were sitting (my granddad`s walking stick is still there mostly buried in drifting sand).  I sink to my knees, start scrambling through the sand, find the burried butt.  Triangulate where everyone was sitting during lunch.  As the tide was licking my toes and my granddad was cursing that it was gone, I pulled the denture out of the sand.  I nonchalantly handed it to him and said "keep this in your mouth"

Summarized by algo

It works well, now let's try to implement some measures between the algorithm summary and the human summary.

# Measurement between summarization output and human summary - Basic distances

First we try some basic distances

In [ ]:
!pip install textdistance

In [ ]:
import textdistance as td

In [ ]:
summary_algo = output 
summary_human = df_no_troll["summary"].iloc[0]

In [ ]:
#Calculate the levenshtein distance between the two summary
#first the Levenshtein distance 
td.levenshtein(summary_algo,summary_human)


190

In [ ]:
#The LZMA distance, which is a Compression based distance
td.lzma_ncd(summary_algo,summary_human)

0.656

In [ ]:
#Token Based distance 
td.jaccard(summary_algo,summary_human)

0.43866171003717475

Let's try those distances on 5 content to see how they do.

In [ ]:
lev = []
lzm = []
jacc = []

df_content = df_no_troll['content'].astype(str)


for i in range(0,5):
  #Summarize of the 5 first content
  text =df_content.iloc[i]
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: " + preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

  summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print ("\n\nSummarized by algo text: ",output)
  print ("\n\nSummarized by human text: \n",df_no_troll.iloc[i]['summary'])

  #Calculating the distance for different methods
  lev.append(td.levenshtein(output,df_no_troll.iloc[i]['summary']))
  lzm.append(td.lzma_ncd(output,df_no_troll.iloc[i]['summary']))
  jacc.append(td.jaccard(output,df_no_troll.iloc[i]['summary']))
  



Summarized by algo text:  my granddad realizes that his partial denture isn't in his mouth. he snuck a ciggy while the family was frolicking in the surf, and buried it on the drive home. we drive back to the beach, tide is coming in. I run to where we were sitting, find the burried butt.


Summarized by human text: 
 gramps lost his teeth at the beach and I found them an hour later minutes before the tide swept them away \n EDIT: formatting


Token indices sequence length is longer than the specified maximum sequence length for this model (824 > 512). Running this sequence through the model will result in indexing errors




Summarized by algo text:  the first week of torture, with bills to think of, I had to stop smoking all together. a friend showed up to smoke some rather top shelf bud, and he's been smoking me out - and has been showing up twice every day and smoking him out, despite my woes 'the past few days'


Summarized by human text: 
 Lost my job, friend now coming by twice a day to smoke me out.


Summarized by algo text:  most major governments actions are to insure that their people have jobs, a decent quality of life, and most importantly the value of their currency both in the short term and the long term. all countries are in constant conflict with each other becuase they are all trying to accomplish the above said goals to the greatest degree possible.


Summarized by human text: 
 The only way you can think Snowden is a traitor is if you are naive and don't know the cause and effect relations that exist that produce the quality of life of people in countries.


Summarized by algo text: 

In [ ]:
lev

[190, 221, 240, 190, 162]

In [ ]:
lzm

[0.656,
 0.6991869918699187,
 0.6530612244897959,
 0.6614173228346457,
 0.6355140186915887]

In [ ]:
jacc

[0.43866171003717475,
 0.21691176470588236,
 0.48717948717948717,
 0.575,
 0.4890829694323144]

After looking at the results of every distance and the perceived distance when reading the summaries we can tell that they are not very accurate for our study

# Trying a word embeding distance

### Importing W2V and Word moving distance

In [ ]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
summary_algo_2 = [w for w in summary_algo.lower().split() if w not in stop_words]
summary_human_2 = [w for w in summary_human.lower().split() if w not in stop_words]

In [ ]:
print(summary_algo_2)
print(summary_human_2)

['granddad', 'realizes', 'partial', 'denture', 'mouth.', 'snuck', 'ciggy', 'family', 'frolicking', 'surf,', 'buried', 'drive', 'home.', 'drive', 'back', 'beach,', 'tide', 'coming', 'in.', 'run', 'sitting,', 'find', 'burried', 'butt.']
['gramps', 'lost', 'teeth', 'beach', 'found', 'hour', 'later', 'minutes', 'tide', 'swept', 'away', '\\n', 'edit:', 'formatting']


In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
#model_gn = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/MasterThesis/GoogleNews-vectors-negative300.bin.gz', binary=True)
model_gn = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/GoogleNews-vectors-negative300.bin.gz', binary= True)

In [ ]:
distance = model_gn.wmdistance(summary_algo_2, summary_human_2)
distance

3.0911760454313804

Now let's try the distance on more content to see if it follows the differences we perceive as humans.

### Summarizing the 5 first content and showing the distance between the two summaries

In [ ]:
df_no_troll = pd.read_csv('/content/gdrive/MyDrive/MasterThesis/reddit300k_without_troll.csv')

In [ ]:
wmd = []


for i in range(0,5):
  #Summarize of the 5 first content
  text =df_content.iloc[i]
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: " + preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

  summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print ("\n\nSummarized by algo text: \n",output)
  print ("\n\nSummarized by human text: \n",df_no_troll.iloc[i]['summary'])

  #Calculating the distance

  distance = model_gn.wmdistance(output, df_no_troll.iloc[i]['summary'])

  print("\nDistance between the summaries : ",distance)
  print("-"*200)

  wmd.append(distance)






Summarized by algo text: 
 recollection of the rape would not prevent the event from happening. if you go back in time and murder the murderist before he raped me, are you following?


Summarized by human text: 
 gramps lost his teeth at the beach and I found them an hour later minutes before the tide swept them away \n EDIT: formatting

Distance between the summaries :  0.7137803001544188
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Summarized by algo text: 
 my girlfriend and I were together for 3 years and bought a house together. we were happy together but she had told me that I was not being as attentive/loving as I should be, which I completely agree with. she is the type of person who needs constant validation, attention and is babied by her parents. now I did my best to show her I love her but no matter what she did she 

So far it seems accurate. 

### Summarizing 10 random content of the dataset

We are going to do it on 10 more content

In [ ]:
import numpy as np

index = np.random.randint(len(df_content), size=10)

In [ ]:
wmd = []

for i in index:
  text =df_content.iloc[i]
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: " + preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

  summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print ("\n\nSummarized by algo text: \n",output)
  print ("\n\nSummarized by human text: \n",df_no_troll.iloc[i]['summary'])

  #Calculating the distance

  distance = model_gn.wmdistance(output, df_no_troll.iloc[i]['summary'])

  print("\nDistance between the summaries : ",distance)

  wmd.append(distance)



Summarized by algo text: 
 two, who the hell doesn't keep their ID with them? it happens so often and I'm perplexed by it. do people not keep them in their wallet?


Summarized by human text: 
 actual advice bit: I agree with Sommiel, "Leave it alone, or be her real friend. Your decision."

Distance between the summaries :  0.9567058888690331


Token indices sequence length is longer than the specified maximum sequence length for this model (654 > 512). Running this sequence through the model will result in indexing errors




Summarized by algo text: 
 two, who the hell doesn't keep their ID with them? it happens so often and I'm perplexed by it. do people not keep them in their wallet?


Summarized by human text: 
 actual advice bit: I agree with Sommiel, "Leave it alone, or be her real friend. Your decision."

Distance between the summaries :  0.9567058888690331


Token indices sequence length is longer than the specified maximum sequence length for this model (675 > 512). Running this sequence through the model will result in indexing errors




Summarized by algo text: 
 n my dad died un-expectantly last christmas and it shook my family to the core. the next day my mom and uncle found his body at his small business he owns, a constant reminder of him not being there any more. it was like that first day all over again. but we all brushed it off, we had to. I stayed strong for my mother, because she had it even tougher, working full time


Summarized by human text: 
 Asked Girl out, got response, want to know if its an excuse or actually want to hang when free

Distance between the summaries :  0.6182457502385841


Summarized by algo text: 
 BF and his ex broke up about 4 years ago. bf is taking a nap and I feel like she's testing me and sizing me up?


Summarized by human text: 
 I told my mom I'm an atheist. She freaks out and threatens to send me away, but my awesome dad is cool with it and calms her down.

Distance between the summaries :  1.0377689936590284


Summarized by algo text: 
 my boyfriend of 3 years has become 

The results are really encouraging. But we have multiple error message saying that the original length of the content is too long for our model. So let's create a clean dataset of 20 content that we will use to benchmark the algorithm and to improve it.

# Creating a subset of the dataset with only 20 contents

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
df_no_troll = pd.read_csv('/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/reddit300k_without_troll.csv')
#df_no_troll = pd.read_csv('/content/gdrive/MyDrive/MasterThesis/reddit300k_without_troll.csv')

In [ ]:
len(df_no_troll)

119101

In [ ]:
df_no_troll = df_no_troll.loc[df_no_troll['content'].apply(lambda x : len(x.split()) < 512 )]

In [ ]:
df_small = df_no_troll.sample(n = 200).reset_index(drop=True)

In [ ]:
df_small.head(10)

,content,subreddit,summary
0,We actually did something very similar to this...,pics,"High school prank, we dropped nearly 60,000 bo..."
1,"Hello Reddit, \n I have been full of emotions ...",relationships,"My boyfriend became so ""cold"" after being few ..."
2,This is by far my favorite story to tell. I\'v...,AskReddit,My friend's first girlfriend had a sugardaddy ...
3,FAKE: blood makes up about 7% of the bodies ma...,WTF,If real (which it is not) His BAC would be 10%...
4,"While it no doubt raises some eyebrows, it's n...",news,"This was an exception, the Air Force wouldn't ..."
5,"At work last week, a patient who is a known sc...",AskReddit,"drunk, schizo affective disorder patient freak..."
6,"My first post, copy/pasted here from [another ...",AskReddit,Devote your full attention and focus to your b...
7,I understand this mindset and see it a lot. Y...,funny,Consider upgrading the slowest component rathe...
8,"In all fairness, people just up and vanished f...",asoiaf,Benjen disappearing is a deliberate and themat...
9,I stand on the side that absolutely loves this...,DestinyTheGame,I wonder if Bungie is working on fixing the Ne...


In [ ]:
#df_small.to_csv (r'/content/gdrive/MyDrive/MasterThesis/reddit20_clean.csv', header = True, index = False)
df_small.to_csv (r'/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/reddit200_clean.csv', header=True, index = False)

# Summarization task on this dataset, trying to reduce the distance

## Loading of libraries and dataset if we want to skip the previous sections 

In [ ]:
!pip install transformers==2.8.0
!pip install torch
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

     |████████████████████████████████| 573kB 7.6MB/s 
     |████████████████████████████████| 133kB 15.2MB/s 
     |████████████████████████████████| 5.6MB 10.1MB/s 
     |████████████████████████████████| 901kB 48.5MB/s 
     |████████████████████████████████| 1.2MB 37.2MB/s 
     |████████████████████████████████| 7.6MB 39.0MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
ERROR: botocore 1.20.83 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install gensim
import gensim
from gensim.models import Word2Vec

In [ ]:
#model_gn = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/MasterThesis/GoogleNews-vectors-negative300.bin.gz', binary=True)
model_gn = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/GoogleNews-vectors-negative300.bin.gz', binary= True)

In [ ]:
df_mediate = pd.read_csv('/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/reddit100_clean.csv')
df_mediate

,content,subreddit,summary
0,"No worries, same thing happened to me with my ...",AdviceAnimals,"Was a gentleman, first time BDSM, first time a..."
1,"Like some people have said here already, math ...",explainlikeimfive,"Physicists started using math as play blocks, ..."
2,right the opposite: whenever a used game is re...,technology,good for companies but extremly bad for gamers...
3,"We had a regular wooden swing set, also not se...",AskReddit,make sure you have open space behind and in fr...
4,"I\'m kind of torn on this article, and the pub...",politics,I entirely agree with the mother that mental h...
...,...,...,...
95,I had a former friend in College who constantl...,AskReddit,Drunken idiot demands drinks and makes a fool ...
96,"First, the way insurance works, you already pa...",politics,you pay for everybody's healthcare whether you...
97,Hey dude. It's all about how hard your try. He...,AskReddit,I got lucky with my first job and then honed m...
98,"For one thing, they would have to engineer new...",AskReddit,"it wouldn't fix anything, and your retarded le..."


## Summariztion step 1 : Full content

In [ ]:
wmd = []

df_content = df_mediate['content'].astype(str)
final_df = pd.DataFrame({'Generated summary':[],'Human Summary':[]})

for i in range(0,len(df_content)):
  text =df_content.iloc[i]
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: " + preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

  summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print ("len original content", len(preprocess_text))
  print ("len summarized content", len(output.strip()))
  print ("\n\nSummarized by algo text: \n",output)
  print ("\n\nSummarized by human text: \n",df_mediate.iloc[i]['summary'])

  final_df = final_df.append({'Generated summary':output,'Human Summary':df_mediate.iloc[i]['summary']}, ignore_index=True)

  #Calculating the distance

  distance = model_gn.wmdistance(output, df_mediate.iloc[i]['summary'])

  print("\nDistance between the summaries : ",distance)
  print("-"*200)

  wmd.append(distance)

len original content 552
len summarized content 305


Summarized by algo text: 
 ex GF and her family chose the most expensive bottle of wine on the list. they chose a fucking French Mommessin, and then went to anal sex for the first time in the week, before dumping her for an hour later in january, she was dragged to the riverside by her mother's mother, who was in love with her ex.


Summarized by human text: 
 Was a gentleman, first time BDSM, first time anal sex, totally worth it

Distance between the summaries :  0.9640082803264814
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
len original content 2097
len summarized content 308


Summarized by algo text: 
 math is very useful in solving problems. n that's pretty simple, but it works really for the simple stuff. it can describe how much pressure is in an oven as it heats up, or 

Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors


len original content 724
len summarized content 194


Summarized by algo text: 
 we had a regular wooden swing set, also not set into the ground, so it would do the same. as we grew older and swung ourselves, we also had to extend the chains on the swings so we could get on.


Summarized by human text: 
 make sure you have open space behind and in front if swing sets.

Distance between the summaries :  0.7655387562485549
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
len original content 2493
len summarized content 449


Summarized by algo text: 
 the inadequacy of mental health services is being highlighted, and the extreme strain that many of these people and their families are under. the mother is simply sharing her experience and comparing her son to Lanza, but neither mental illnesses themselves nor individuals with them are inte

Token indices sequence length is longer than the specified maximum sequence length for this model (690 > 512). Running this sequence through the model will result in indexing errors


len original content 440
len summarized content 153


Summarized by algo text: 
 daaaammmnn has a hard time recommending the movie to others without lots of warnings. it's an amazing piece of cinema and stands out within the art form.


Summarized by human text: 
 the scene is bad but you must watch it in full context of the whole movie to understand it's relevance.

Distance between the summaries :  0.6813770060816999
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
len original content 2529
len summarized content 281


Summarized by algo text: 
 'everyone should learn how to play against it' n fk that, this is ranked, learn on your own time. you are assuming FAR too much about the enemy and the rest of your team. Especially since most of the time when I see this they let the OP picks through for lanes they aren't even in.


Summarized

...

len original content 1361
len summarized content 355


Summarized by algo text: 
 nNobody ever made me read books at a young age, or even how to read in general. the first word I ever read, it was near the end of second grade, about two years after most folks learn, and teacher asked me to write the word - "Baby?" "that's right!" 'nI still can not read quite as well as I would like, but the problem I had where I forget what I read is


Summarized by human text: 
 Learning to read is fucking hard if you're not a kid.

Distance between the summaries :  1.0428237256230282
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
len original content 2081
len summarized content 215


Summarized by algo text: 
 n I'm 26 year old male male living in a new town. in my 11 months here I've attached myself to several bands as lead singer/guitarist. he's b

Token indices sequence length is longer than the specified maximum sequence length for this model (718 > 512). Running this sequence through the model will result in indexing errors


len original content 1576
len summarized content 154


Summarized by algo text: 
 each planet should have a Guardian area with everything you need. n saves load screen times just to get something from the vault or handing in bounty etc.


Summarized by human text: 
 Not only would it reduce load times but could be used as a great community focused 16 man fire team events. \n EDIT1: Spelling;\nEDIT2: Added 5) & 6)

Distance between the summaries :  0.8071057924885813
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
len original content 2608
len summarized content 140


Summarized by algo text: 
 if any of you girls have had this and had it removed, how was it? what was the procedure/recovery like? a gynecologist will be able to help.


Summarized by human text: 
 Bean sized lump in my vagina making sex completely unbearable. Seeing gyno 

Token indices sequence length is longer than the specified maximum sequence length for this model (686 > 512). Running this sequence through the model will result in indexing errors


len original content 1010
len summarized content 256


Summarized by algo text: 
 a hardcore rp setting that is well moderated and not sex play focused is essentially impossible but is so much more rewarding than grinding to level cap just to chase item skins or meaningless other "prestige" type achievements no one but you ever notices.


Summarized by human text: 
 it's all empty time wastage, even at it's best

Distance between the summaries :  0.9403195333586706
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
len original content 2476
len summarized content 365


Summarized by algo text: 
 a friend of mine and her ex-bf asked me to meet her downtown in college. she called me and asked her to pick her up in front of school and she didn't want to go anywhere with her anymore but she wanted to make her jealous that she's out having goo

In [ ]:
final_df.to_csv(r'/content/gdrive/MyDrive/UPM_ALUMNOS_2021/MasterThesis/models/pretrainedpred100.csv', header=True, index = False)

In [ ]:
mean_score_1 = np.mean(np.array(wmd))
mean_score_1

0.6701409701067348

In [ ]:
with open('/content/gdrive/My Drive/UPM_ALUMNOS_2021/MasterThesis/Results.txt', 'w') as f:
  f.write("Mean Distance between summaries with full content as input : %s " % mean_score_1)

## With keyword extraction 

Now that we have a mean WMD for the small dataset we are going to try to improve it we different methods. Let's start with keyword extraction.

In [ ]:
!pip install summa

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for summa: filename=summa-1.2.0-cp37-none-any.whl size=54411 sha256=19f65df232cb16f9cab58ab539c1374d6562e2aa0545761a57403cf2e23761f8
  Stored in directory: /root/.cache/pip/wheels/6a/09/68/e2f2861c01d86407c3fa5220826ed7eed2abaa56b001be5970
Successfully built summa


In [ ]:
from summa import keywords

wmd_2 =[]

for i in range(0,len(df_content)):
  text = df_small.iloc[i]['content']
  preprocess_text = keywords.keywords(text,ratio=1,)
  preprocess_text = preprocess_text.strip().replace("\n"," ")

  t5_prepared_Text = "summarize: " + preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

  summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print ("len original content", len(text.strip()))
  print ("len summarized content", len(output.strip()))
  print ("\n\nSummarized by algo text: \n",output)
  print ("\n\nSummarized by human text: \n",df_small.iloc[i]['summary'])

  #Calculating the distance
  summary_algo = [w for w in output.lower().split() if w not in stop_words]
  summary_human = [w for w in df_small.iloc[i]['summary'].lower().split() if w not in stop_words]
  

  
  distance = model_gn.wmdistance(summary_algo, summary_human)

  print("\nDistance between the summaries : ",distance)
  print("-"*200)

  wmd_2.append(distance)

len original content 295
len summarized content 183


Summarized by algo text: 
 hypothetically lets say happening happen happen happens happens happen. let's say that happening happening happens. the real thing is a hypothetical way to let say it happened happen.


Summarized by human text: 
 I was raped, went back in time to kill my rapist before I was raped, did the rape still happen?

Distance between the summaries :  3.1574654608229875
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
len original content 1458
len summarized content 333


Summarized by algo text: 
 ago weeks love loved long time lurker attentive validation attention needed constant non existent fully accepts cool head sex life completely agree history little finally decided n half feel thing thing. i'd love love a longtime lurking - he's the ess : it is all u... it 

In [ ]:
mean_score_2 = np.mean(np.array(wmd_2))
mean_score_2

0.7238098543172934

The results are pretty bad. During keyword extraction we lose a lot of the informations of the original content. Just by reading the keyword extraction of the original content we can tell that it is not going to work. 

#Summarizing a text already summarized to see if it improves

First : Summarization of the content

In [ ]:
text =df_small.iloc[1]['content']
preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: " + preprocess_text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("Original content : \n",preprocess_text)
print ("\nSummarized by algo text: \n",output)
print("\nSummary given by the author \n",df_small.iloc[1]['summary'])

Original content : 
 Long time lurker and finally decided to post something. \n A little history, my girlfriend and I were together for 3 years and bought a house together about a year and half ago. We were happy together but she had told me that I was not being as attentive/loving as I should be, which I completely agree with. Our sex life was lacking but not like it was non existent and no matter what I did she would just attribute it to me only trying to make her happy and not meaning it? She is the type of person who needs constant validation, attention and is babied by her parents. Now I did my best to show her I love her but no matter what I did she would just brush it off that I didn\'t show it enough. \n 2 weeks ago I caught her cheating on me with a guy who she had met up with and confronted her about it. It was a 1 time thing and her reasoning was "I didn\'t feel loved". It was about 2 minutes of fighting but in the end I decided I needed some time to think about this and I m

Second : Summarization of the previous output

In [ ]:
text =output
preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: " + preprocess_text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
  
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("Original content : \n",preprocess_text)
print ("\nSummarized by algo text: \n",output)
print("\nSummary given by the author \n",df_small.iloc[1]['summary'])

Original content : 
 my girlfriend and I were together for 3 years and bought a house together. we were happy together but she had told me that I was not being as attentive/loving as I should be, which I completely agree with. she is the type of person who needs constant validation, attention and is babied by her parents. now I did my best to show her I love her but no matter what she did she would just brush it off that she didn't show it

Summarized by algo text: 
 my girlfriend and we were together for 3 years and bought a house together. she had told me that she was not being as attentive/loving as she should be, which she completely agrees with.

Summary given by the author 
 Girlfriend of 3 years cheated on me because she said she didn't get enough affection


This is the final result : 


First summary:
> my girlfriend and I were together for 3 years and bought a house together. we were happy together but she had told me that I was not being as attentive/loving as I should be, which I completely agree with. she is the type of person who needs constant validation, attention and is babied by her parents. now I did my best to show her I love her but no matter what she did she would just brush it off that she didn't show it

Second summary:

> my girlfriend and we were together for 3 years and bought a house together. she had told me that she was not being as attentive/loving as she should be, which she completely agrees with.

Summary given by the author:


> Girlfriend of 3 years cheated on me because she said she didn't get enough affection


In [ ]:
summary_algo_1 = [w for w in preprocess_text.lower().split() if w not in stop_words]
summary_algo_2 = [w for w in output.lower().split() if w not in stop_words]
summary_human = [w for w in df_small.iloc[1]['summary'].lower().split() if w not in stop_words]

In [ ]:
distance_1 = model_gn.wmdistance(summary_algo_1, summary_human)
distance_1

2.6850243163430108

In [ ]:
distance_2 = model_gn.wmdistance(summary_algo_2, summary_human)
distance_2

2.0967883763362485

We have reduced the distance, but when reading we have also lost some precious informations : 
>  as attentive/loving as I should be, which I completely agree with. 

Turns into : 

> attentive/loving as she should be, which she completely agrees with.

# Fine tuning 

We do the finetuning in another notebook to keep things clean. See 'T5_finetune' notebook.